In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
df_encode = pd.get_dummies(df, columns=["home_ownership","verification_status", "issue_d", "pymnt_plan","initial_list_status","next_pymnt_d", "application_type","hardship_flag", "debt_settlement_flag"])
X = df_encode.drop(columns=['loan_status'], axis=1)

# Create our target
y = df['loan_status']

In [7]:
pd.set_option('display.max_rows', None)
X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
loan_amnt,68817.0,16677.594562,10277.348590,1000.00,9000.0000,15000.000,24000.0000,4.000000e+04
int_rate,68817.0,0.127718,0.048130,0.06,0.0881,0.118,0.1557,3.084000e-01
installment,68817.0,480.652863,288.062432,30.89,265.7300,404.560,648.1000,1.676230e+03
annual_inc,68817.0,88213.705210,115579.958439,40.00,50000.0000,73000.000,104000.0000,8.797500e+06
dti,68817.0,21.778153,20.199244,0.00,13.8900,19.760,26.6600,9.990000e+02
delinq_2yrs,68817.0,0.217766,0.718367,0.00,0.0000,0.000,0.0000,1.800000e+01
inq_last_6mths,68817.0,0.497697,0.758122,0.00,0.0000,0.000,1.0000,5.000000e+00
open_acc,68817.0,12.587340,6.022869,2.00,8.0000,11.000,16.0000,7.200000e+01
pub_rec,68817.0,0.126030,0.336797,0.00,0.0000,0.000,0.0000,4.000000e+00
revol_bal,68817.0,17604.142828,21835.880400,0.00,6293.0000,12068.000,21735.0000,5.871910e+05


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
# import sklearn train_test_split
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing subsets 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1 ,
                                                   stratify = y)
# Counter module confirms the imbalance in the training set.
Counter(y_train)

Counter({'low_risk': 51352, 'high_risk': 260})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier

RFClassifier = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
RFClassifier.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [11]:
## make predictions and calucate balanced accuracy score
y_pred = RFClassifier.predict(X_test)
bac_Rand_Forest = balanced_accuracy_score(y_test, y_pred)
print('The balanced accuracy score for BalancedRandomForestClassifier is', bac_Rand_Forest, '\n')

# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_Rand_Forest = confusion_matrix(y_test, y_pred)
print('The confusion matrix for BalancedRandomForestClassifier is \n', cm_Rand_Forest, '\n')

# Print the imbalanced classification report
imb_classRep_Rand_Forest = classification_report_imbalanced(y_test, y_pred)
print('The imbalanced classification report for BalancedRandomForestClassifier is \n', imb_classRep_Rand_Forest)

The balanced accuracy score for BalancedRandomForestClassifier is 0.7877672625306695 

The confusion matrix for BalancedRandomForestClassifier is 
 [[   58    29]
 [ 1560 15558]] 

The imbalanced classification report for BalancedRandomForestClassifier is 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.67      0.91      0.07      0.78      0.59        87
   low_risk       1.00      0.91      0.67      0.95      0.78      0.62     17118

avg / total       0.99      0.91      0.67      0.95      0.78      0.62     17205



In [12]:
# List the features sorted in descending order by feature importance
sorted(zip(RFClassifier.feature_importances_, X.columns), reverse=True)

[(0.07376667607601396, 'total_rec_prncp'),
 (0.06390324452717588, 'total_rec_int'),
 (0.06073336071656837, 'total_pymnt_inv'),
 (0.05811195697921674, 'total_pymnt'),
 (0.04951778391272079, 'last_pymnt_amnt'),
 (0.02458051789018817, 'int_rate'),
 (0.020398879691407974, 'out_prncp'),
 (0.018625883307901298, 'dti'),
 (0.018378884967316288, 'max_bal_bc'),
 (0.017480030880564042, 'issue_d_Jan-2019'),
 (0.01698703048268034, 'installment'),
 (0.01661321625916749, 'annual_inc'),
 (0.01623198470200134, 'bc_util'),
 (0.015821923028819504, 'mths_since_recent_inq'),
 (0.015385686591272854, 'total_bc_limit'),
 (0.015337070167127763, 'out_prncp_inv'),
 (0.01506649550412446, 'avg_cur_bal'),
 (0.014926455495755147, 'issue_d_Mar-2019'),
 (0.014924489307566175, 'revol_bal'),
 (0.014583210727445232, 'mo_sin_old_rev_tl_op'),
 (0.01428231366275729, 'total_bal_ex_mort'),
 (0.014238473969302518, 'tot_cur_bal'),
 (0.014067369111282934, 'mo_sin_old_il_acct'),
 (0.01351439379612201, 'total_bal_il'),
 (0.0133438

### Easy Ensemble AdaBoost Classifier

In [13]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=1)
eec.fit(X_train, y_train) 

EasyEnsembleClassifier(random_state=1)

In [14]:
## make predictions and calucate balanced accuracy score
y_pred = eec.predict(X_test)
bac_eec = balanced_accuracy_score(y_test, y_pred)
print('The balanced accuracy score for EasyEnsembleClassifier is', bac_eec, '\n')

# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_eec = confusion_matrix(y_test, y_pred)
print('The confusion matrix for EasyEnsembleClassifier is \n', cm_eec, '\n')

# Print the imbalanced classification report
imb_eec = classification_report_imbalanced(y_test, y_pred)
print('The imbalanced classification report for EasyEnsembleClassifier is \n', imb_eec)

The balanced accuracy score for EasyEnsembleClassifier is 0.9197970678173006 

The confusion matrix for EasyEnsembleClassifier is 
 [[   78     9]
 [  975 16143]] 

The imbalanced classification report for EasyEnsembleClassifier is 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.90      0.94      0.14      0.92      0.84        87
   low_risk       1.00      0.94      0.90      0.97      0.92      0.85     17118

avg / total       0.99      0.94      0.90      0.97      0.92      0.85     17205

